### Ładuję dane Movies

In [0]:
from pyspark.sql.functions import *

moviesDf = spark.read.format("csv") \
              .option("header", "true") \
              .option("inferSchema", "true") \
              .load("dbfs:/FileStore/tables/Files/movies.csv")

moviesDf.printSchema()
display(moviesDf.limit(5))

root
 |-- imdb_title_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- date_published: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- language: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- production_company: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- description: string (nullable = true)
 |-- avg_vote: string (nullable = true)
 |-- votes: integer (nullable = true)
 |-- budget: string (nullable = true)
 |-- usa_gross_income: string (nullable = true)
 |-- worlwide_gross_income: string (nullable = true)
 |-- metascore: double (nullable = true)
 |-- reviews_from_users: double (nullable = true)
 |-- reviews_from_critics: double (nullable = true)



imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey Depew",The adventures of a female reporter in the 1890s.,05.wrz,154,null,null,null,null,1.0,2.0
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,26.12.1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson",True story of notorious Australian outlaw Ned Kelly (1855-80).,06.sty,589,$ 2250,null,null,null,7.0,7.0
tt0001892,Den sorte drøm,Den sorte drøm,1911,19.08.1911,Drama,53,"Germany, Denmark",null,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen","Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",05.sie,188,null,null,null,null,5.0,2.0
tt0002101,Cleopatra,Cleopatra,1912,13.11.1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, Miss Robson, Helene Costello, Charles Sindelar, Mr. Howard, James R. Waite, Mr. Osborne, Harry Knowles, Mr. Paul, Mr. Brady, Mr. Corker",The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.,05.lut,446,$ 45000,null,null,null,25.0,3.0
tt0002130,L'Inferno,L'Inferno,1911,06.03.1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de Liguoro, Pier Delle Vigne, Augusto Milla, Attilio Motta, Emilise Beretta",Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.,7.0,2237,null,null,null,null,31.0,14.0


### Zadanie 1
- Użyj poniższe funkcje Nulls, fill, explode, drop, regexp_replace, regexp_extract, ifnull, nullIf, replace, array_contains.
- Użyj 3 funkcji agregujących (które według Ciebie są najciekawsze).

In [0]:
from pyspark.sql.functions import col, expr, regexp_replace, regexp_extract, when, array_contains, explode, avg, count, sum

# fill
moviesDf = moviesDf.na.fill({"language": "Brak danych"})

# explode
moviesDf = moviesDf.withColumn("title", explode(expr("split(title, ' ')"))) 

# drop
moviesDf = moviesDf.drop("metascore")

# regexp_replace
moviesDf = moviesDf.withColumn("clean_budget", regexp_replace("budget", "[$,]", "").cast("double"))

# regexp_extract
moviesDf = moviesDf.withColumn("regexp_extract", regexp_extract(col("actors"), "(\d+)", 1).cast("int"))

# coalesce (ifnull niedostępny)
moviesDf = moviesDf.withColumn("director", coalesce(col("director"), lit("Brak danych")))

# when (nullif niedostępny)
moviesDf = moviesDf.withColumn("votes", when(trim(col("votes")) == "Brak danych", None).otherwise(col("votes")))

# replace
moviesDf = moviesDf.withColumn("landmark", regexp_replace(col("country"), "Italy", "Włochy"))

In [0]:
# Funkcje agregujące
df2 = moviesDf.groupBy("country").agg(
    count("reviews_from_users").alias("liczba_review"),
    sum("duration").alias("suma_czasu_trwania_filmow"),
    avg("duration").alias("srednia_dlugosc_filmow")
) 
display(df2.limit(20))

country,liczba_review,suma_czasu_trwania_filmu,srednia_dlugosc_filmu
"UK, Spain",85,8319,97.87058823529412
"India, Germany",7,835,119.28571428571429
"USA, France, UK",20,2061,103.05
Russia,1150,181375,98.25297941495124
"UK, France",260,26999,103.04961832061069
"West Germany, USA",88,8857,100.64772727272727
"France, Italy, East Germany",2,420,210.0
"Greece, France, USA",1,115,115.0
"USA, Japan",314,32679,102.44200626959248
"Italy, France, Germany",24,3189,106.3


### Zadanie 2
- Stwórz 2 funkcje UDF do wybranego zestawu danych, wymyśl co mają robić w kontekście wybranych danych. Jedna funkcja z dekoratorem @pandas_udf druga standardowa.
  - a.	Jedna funkcja działające na typach liczbowych: int, double 
  - b.	Jedna funkcja na string

In [0]:
#funkcja czy budzet filmu przekracza 10k
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

#działa na double (budget) i zwraca tekst
def budget_category(budget):
    if budget is None:
        return "Unknown"
    elif budget > 10_000:
        return "High"
    elif budget > 2_000:
        return "Medium"
    else:
        return "Low"

budget_category_udf = udf(budget_category, StringType())

moviesDf = moviesDf.withColumn("budget_category", budget_category_udf(col("clean_budget")))
display(moviesDf.select("title", "clean_budget", "budget_category").limit(10))

title,clean_budget,budget_category
Miss,null,Unknown
Jerry,null,Unknown
The,2250.0,Medium
Story,2250.0,Medium
of,2250.0,Medium
the,2250.0,Medium
Kelly,2250.0,Medium
Gang,2250.0,Medium
Den,null,Unknown
sorte,null,Unknown


In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd

#działa na stringach i zwraca int (długość tytułu)
@pandas_udf(IntegerType())
def title_length(title_col: pd.Series) -> pd.Series:
    return title_col.str.len()

moviesDf = moviesDf.withColumn("title_length", title_length(col("title")))
display(moviesDf.select("title", "title_length").limit(10))

title,title_length
Miss,4
Jerry,5
The,3
Story,5
of,2
the,3
Kelly,5
Gang,4
Den,3
sorte,5
